#Demonstration: Building an NLP Pipeline for Multilingual Tweet Cleaning

##Scenario:

- Alex, an NLP engineer at a social media company, receives a messy dataset of multilingual tweets filled with emojis, emoticons, hashtags, mentions, and noise. Since the goal is to analyze only English tweets, manually cleaning and filtering them would be too time-consuming. To streamline this, she builds an automated NLP pipeline that detects language, filters for English, removes noise, lowercases text, eliminates stopwords, tokenizes content, and applies techniques like BPE, one-hot encoding, and TF-IDF—turning raw tweets into clean, structured data ready for analysis.

##Import Libraries

In [4]:
!pip3 install nltk langdetect spacy ftfy contractions emoji tokenizers

In [5]:
# Core Libraries
import pandas as pd
import re
import string
import unicodedata
import ftfy
import contractions

# Language & Tokenization
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
import spacy

# Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

# Emojis & Emoticons
import emoji

# Byte Pair Encoding (BPE)
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Download resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Load SpaCy
nlp = spacy.load("en_core_web_sm")
print("✅ Libraries imported and models downloaded.")

✅ Libraries imported and models downloaded.


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/varuniexpress/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/varuniexpress/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/varuniexpress/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


##Load and Show Dataset

In [6]:
df = pd.read_csv("tweets_dataset.csv")
print("📄 Sample data:\n", df.head())

📄 Sample data:
   username location  gender  age  \
0   user_1    India  Female   43   
1   user_2   France    Male   26   
2   user_3  Germany   Other   30   
3   user_4  Germany  Female   38   
4   user_5    Japan  Female   39   

                                           tweet  
0  Can't believe this happened... lol :D #fail 😅  
1                                     信じられない… 😭💔  
2                 Das ist fantastisch 😍💯! #Liebe  
3  Can't believe this happened... lol :D #fail 😅  
4                  ¡Esto es perfecto! 😊💃 #fiesta  


##Detect Languages

In [7]:
df['language'] = df['tweet'].apply(lambda x: detect(x))
print("Language Count:\n", df['language'].value_counts())

Language Count:
 language
de    14
id    14
fr    13
pt    13
ja    12
en     8
ru     8
hi     7
es     6
ur     4
ar     1
Name: count, dtype: int64


##Filter English Tweets

In [8]:
df_en = df[df['language'] == 'en'].reset_index(drop=True)
print("Filtered English Tweets:\n", df_en[['tweet']].head())

Filtered English Tweets:
                                            tweet
0  Can't believe this happened... lol :D #fail 😅
1  Can't believe this happened... lol :D #fail 😅
2  Can't believe this happened... lol :D #fail 😅
3        I love this! 😍 Soooo good!!! #awesome 😊
4  Can't believe this happened... lol :D #fail 😅


##Lowercasing

In [9]:
df_en['cleaned'] = df_en['tweet'].apply(lambda x: x.lower())
print("Lowercased Output :\n", df_en['cleaned'].head())

# Output

Lowercased Output :
 0    can't believe this happened... lol :d #fail 😅
1    can't believe this happened... lol :d #fail 😅
2    can't believe this happened... lol :d #fail 😅
3          i love this! 😍 soooo good!!! #awesome 😊
4    can't believe this happened... lol :d #fail 😅
Name: cleaned, dtype: object


##Stopword Removal

In [10]:
stop_words = set(stopwords.words('english'))
df_en['cleaned'] = df_en['cleaned'].apply(
    lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words])
)
print("Post stopword removal:\n", df_en['cleaned'].head())

Post stopword removal:
 0    ca n't believe happened ... lol : # fail 😅
1    ca n't believe happened ... lol : # fail 😅
2    ca n't believe happened ... lol : # fail 😅
3         love ! 😍 soooo good ! ! ! # awesome 😊
4    ca n't believe happened ... lol : # fail 😅
Name: cleaned, dtype: object


##Tokenization

In [11]:
df_en['tokens'] = df_en['cleaned'].apply(word_tokenize)
print("Tokens:\n", df_en['tokens'].head())

# Output

Tokens:
 0    [ca, n't, believe, happened, ..., lol, :, #, f...
1    [ca, n't, believe, happened, ..., lol, :, #, f...
2    [ca, n't, believe, happened, ..., lol, :, #, f...
3    [love, !, 😍, soooo, good, !, !, !, #, awesome, 😊]
4    [ca, n't, believe, happened, ..., lol, :, #, f...
Name: tokens, dtype: object


In [12]:
# Stemming 

from nltk.stem import PorterStemmer

porter = PorterStemmer()
df_en['Porter_Stem'] = df_en['cleaned'].apply(
    lambda text: " ".join([porter.stem(word) for word in word_tokenize(text.lower())])
)
df_en[['cleaned', 'Porter_Stem']].head(6)



,cleaned,Porter_Stem
0,ca n't believe happened ... lol : # fail 😅,ca n't believ happen ... lol : # fail 😅
1,ca n't believe happened ... lol : # fail 😅,ca n't believ happen ... lol : # fail 😅
2,ca n't believe happened ... lol : # fail 😅,ca n't believ happen ... lol : # fail 😅
3,love ! 😍 soooo good ! ! ! # awesome 😊,love ! 😍 soooo good ! ! ! # awesom 😊
4,ca n't believe happened ... lol : # fail 😅,ca n't believ happen ... lol : # fail 😅
5,love ! 😍 soooo good ! ! ! # awesome 😊,love ! 😍 soooo good ! ! ! # awesom 😊


In [13]:
# Lemmatization
import spacy

nlp_spacy = spacy.load("en_core_web_sm")
df_en['cleaned'] = df_en['cleaned'].apply(
    lambda text: " ".join([token.lemma_ for token in nlp_spacy(text)])
)
df_en['cleaned'].head()



0    can not believe happen ... lol : # fail 😅
1    can not believe happen ... lol : # fail 😅
2    can not believe happen ... lol : # fail 😅
3        love ! 😍 soooo good ! ! ! # awesome 😊
4    can not believe happen ... lol : # fail 😅
Name: cleaned, dtype: object

##Byte-Pair Encoding (BPE)

In [14]:
# Prepare file for BPE training
with open("bpe_corpus.txt", "w", encoding="utf-8") as f:
    for text in df_en['cleaned']:
        f.write(text + "\n")

# Train and tokenize using BPE
tokenizer = Tokenizer(BPE())
trainer = BpeTrainer(special_tokens=["<unk>"])
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train(files=["bpe_corpus.txt"], trainer=trainer)
df_en['bpe_tokens'] = df_en['cleaned'].apply(lambda x: tokenizer.encode(x).tokens)

print("🧠 BPE token sample:\n", df_en['bpe_tokens'].head())




🧠 BPE token sample:
 0    [can, not, believe, happen, ..., lol, :, #, fa...
1    [can, not, believe, happen, ..., lol, :, #, fa...
2    [can, not, believe, happen, ..., lol, :, #, fa...
3    [love, !, 😍, soooo, good, !, !, !, #, awesome, 😊]
4    [can, not, believe, happen, ..., lol, :, #, fa...
Name: bpe_tokens, dtype: object


##One-Hot Encoding

In [15]:
# Flatten all words into one list
all_words = list(set([word for tokens in df_en['tokens'] for word in tokens]))

# Create encoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit([[word] for word in all_words])

# Encode first tweet for demo
encoded_sample = encoder.transform([[w] for w in df_en['tokens'][0]])
print("🎯 One-Hot for 1st Tweet (shape):", encoded_sample.shape)

🎯 One-Hot for 1st Tweet (shape): (10, 17)


##TF-IDF Vectorization

In [16]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df_en['cleaned'])
print("📈 TF-IDF Shape:", tfidf_matrix.shape)

📈 TF-IDF Shape: (8, 10)


##Remove Noise (URLs, Mentions, Hashtags, Punctuation)

In [17]:
def remove_noise(text):
    text = re.sub(r"http\S+|www\S+", "", text)      # URLs
    text = re.sub(r"@\w+", "", text)                # Mentions
    text = re.sub(r"#\w+", "", text)                # Hashtags
    text = re.sub(rf"[{re.escape(string.punctuation)}]", "", text)  # Punctuation
    return text

df_en['cleaned'] = df_en['cleaned'].apply(remove_noise)
print("Post noise removal:\n", df_en['cleaned'].head())

Post noise removal:
 0    can not believe happen  lol   fail 😅
1    can not believe happen  lol   fail 😅
2    can not believe happen  lol   fail 😅
3        love  😍 soooo good     awesome 😊
4    can not believe happen  lol   fail 😅
Name: cleaned, dtype: object


##Remove Emojis

In [18]:
def remove_emojis(text):
    return emoji.replace_emoji(text, replace='')

df_en['cleaned'] = df_en['cleaned'].apply(remove_emojis)
print("Emojis removed:\n", df_en['cleaned'].head())

Emojis removed:
 0    can not believe happen  lol   fail 
1    can not believe happen  lol   fail 
2    can not believe happen  lol   fail 
3         love   soooo good     awesome 
4    can not believe happen  lol   fail 
Name: cleaned, dtype: object


##Remove Emoticons

In [19]:
emoticons = {":)", ":(", ":D", ":-)", ":-(", ":'(", ":-D", ";)", ":P", ":-P"}

def remove_emoticons(text):
    return ' '.join([word for word in text.split() if word not in emoticons])

df_en['cleaned'] = df_en['cleaned'].apply(remove_emoticons)
print("Emoticons removed:\n", df_en['cleaned'].head())

Emoticons removed:
 0    can not believe happen lol fail
1    can not believe happen lol fail
2    can not believe happen lol fail
3            love soooo good awesome
4    can not believe happen lol fail
Name: cleaned, dtype: object


##Handle Contractions

In [20]:
df_en['cleaned'] = df_en['cleaned'].apply(lambda x: contractions.fix(x))
print("Contractions handled:\n", df_en['cleaned'].head())

Contractions handled:
 0    can not believe happen lol fail
1    can not believe happen lol fail
2    can not believe happen lol fail
3            love soooo good awesome
4    can not believe happen lol fail
Name: cleaned, dtype: object


##Normalization (Accents, Spaces)

In [21]:
def normalize_text(text):
    text = unicodedata.normalize('NFKC', text)  # Unicode normalization
    text = ftfy.fix_text(text)  # Fix messy text
    return text

df_en['cleaned'] = df_en['cleaned'].apply(normalize_text)
print("🧽 Normalized text:\n", df_en['cleaned'].head())

🧽 Normalized text:
 0    can not believe happen lol fail
1    can not believe happen lol fail
2    can not believe happen lol fail
3            love soooo good awesome
4    can not believe happen lol fail
Name: cleaned, dtype: object


##Unicode Normalization

In [22]:
def unicode_normalize(text):
    return ''.join(c for c in unicodedata.normalize('NFD', text)
                   if unicodedata.category(c) != 'Mn')

df_en['cleaned'] = df_en['cleaned'].apply(unicode_normalize)
print("Unicode normalized:\n", df_en['cleaned'].head())

Unicode normalized:
 0    can not believe happen lol fail
1    can not believe happen lol fail
2    can not believe happen lol fail
3            love soooo good awesome
4    can not believe happen lol fail
Name: cleaned, dtype: object


##Display Final Result

In [23]:
print("✅ Final Processed Output Sample:\n", df_en[['tweet', 'cleaned', 'tokens', 'bpe_tokens']].head())

✅ Final Processed Output Sample:
                                            tweet  \
0  Can't believe this happened... lol :D #fail 😅   
1  Can't believe this happened... lol :D #fail 😅   
2  Can't believe this happened... lol :D #fail 😅   
3        I love this! 😍 Soooo good!!! #awesome 😊   
4  Can't believe this happened... lol :D #fail 😅   

                           cleaned  \
0  can not believe happen lol fail   
1  can not believe happen lol fail   
2  can not believe happen lol fail   
3          love soooo good awesome   
4  can not believe happen lol fail   

                                              tokens  \
0  [ca, n't, believe, happened, ..., lol, :, #, f...   
1  [ca, n't, believe, happened, ..., lol, :, #, f...   
2  [ca, n't, believe, happened, ..., lol, :, #, f...   
3  [love, !, 😍, soooo, good, !, !, !, #, awesome, 😊]   
4  [ca, n't, believe, happened, ..., lol, :, #, f...   

                                          bpe_tokens  
0  [can, not, believe, happen, .

##Save Cleaned Dataset

In [24]:
df_en.to_csv("cleaned_multilingual_tweets.csv", index=False)
print("📁 Cleaned dataset saved as 'cleaned_multilingual_tweets.csv'")


📁 Cleaned dataset saved as 'cleaned_multilingual_tweets.csv'
